# Sentinel Data

In [1]:
import mlstac

mlstac.download(
    snippet="isp-uv-es/CloudSEN12Plus", path="data", split="test", quiet=True
)  # use "all" to download the entire dataset
# Load the ML-STAC collection
ds = mlstac.load(snippet="data/main.json")

In [ ]:
subset = ds.metadata[
    (ds.metadata["split"] == "test")
    & (ds.metadata["label_type"] == "high")
    & (ds.metadata["proj_shape"] == 509)
][:10]
datacube = mlstac.get_data(dataset=subset, quiet=True)
datacube.shape

In [ ]:
# Plot the first datapoint

import matplotlib.pyplot as plt
import numpy as np

datapoint = datacube[0]
datapoint_rgb = np.moveaxis(datapoint[[3, 2, 1]], 0, -1) / 5000
fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(datapoint_rgb)
ax[0].set_title("RGB")
ax[1].imshow(datapoint[13], cmap="gray")
ax[1].set_title("Human label")
ax[2].imshow(datapoint[14], cmap="gray")
ax[2].set_title("UnetMobV2 v1.0")
plt.show()

In [ ]:
# Validation
datapoint = datacube[0]
datapoint_rgb = datapoint[[1, 2, 3, 7, 10], ...].astype(np.float32)  # BGRNIR
datapoint.shape, datapoint_rgb.shape

# Validation

In [ ]:
import torch
import pytorch_lightning as pl
from pytorchfi import core as pfi_core

from utils.utils import get_parser, parse_args
from utils.data.data_module import CoreDataModule
from utils.models.ghostnetv2 import ghostnetv2
from utils.models.LightningModelWrapper import ModelWrapper

torch.set_float32_matmul_precision("high")
pl.seed_everything(0, workers=True)
torch.backends.cudnn.benchmark = True

parser, config_parser = get_parser()
args = parse_args(parser, config_parser)

In [ ]:
# Define the datamodule
datamodule = CoreDataModule(args, batch_size=128)

# Define the trainer
trainer = pl.Trainer(
    max_epochs=100, callbacks=None, accelerator="auto", precision="16-mixed"
)

# Define the model
model = ghostnetv2(
    num_classes=0,
    error_model="single",
    inject_p=0.0001,
    inject_epoch=0,
    ckpt="ckpt/GN_SSL_280.pt",
)

x = torch.randn(1, 5, 512, 512)
y, intermediates = model(x)
print(y.shape)
print(intermediates[-1].shape)

model = ModelWrapper(model=model)

pfi_model = pfi_core.fault_injection(
    model.cuda(),
    1,
    input_shape=[5, 512, 512],
    layer_types=[
        # torch.nn.Conv1d,
        torch.nn.Conv2d,
        # torch.nn.Conv3d,
        torch.nn.BatchNorm1d,
        torch.nn.BatchNorm2d,
        torch.nn.BatchNorm3d,
        # torch.nn.Linear,
    ],
    use_cuda=True,
)
pfi_model.print_pytorchfi_layer_summary()
pass

Blocks (12 to toggle): 
* ConvBnAct ✅
* 9 Blocks ✅ ( consisting of 16 GhostBottleneckV2 sub-blocks) ❌
* ConvBnAct ✅
* AdaptiveAvgPool2d ❌
* ConvAct ✅
* Linear ❌


Block Types (2 to toggle):
* Conv2d ✅
* BatchNorm2d ✅
* Linear ❌

inject_index (259 in total):
* 137 Conv2d ✅
* 122 BatchNorm2d ✅

In [ ]:
import csv

csv_file = open("ckpt/layers.csv", mode="r")
csv_reader = csv.reader(csv_file)
layers = [row[0] for row in csv_reader]

i = 100
layer_type = layers[i]
print(layer_type)

In [ ]:
# Start the training
_ = trainer.validate(model=model, datamodule=datamodule)
print(intermediates[-1].shape)

In [60]:
inj_layers = []
idx = 0
for i, (name, desc) in enumerate(model.model.named_modules()):
    n = name
    t = str(type(desc)).split('.')[-1].split("'")[0]
    if t in ["Conv2d", "BatchNorm2d"]:
        inj_layers.append((idx, t, i, n))
        idx += 1

In [ ]:
inj_layers

In [62]:
csv.writer(open("ckpt/layers_info.csv", mode="w")).writerows(inj_layers)

In [ ]:
weights = torch.load("ckpt/GN_SSL_280.pt")
weights.keys()

# Validation Results

In [ ]:
import json, csv
from utils.utils import get_parser, parse_args

parser, config_parser = get_parser()
args = parse_args(parser, config_parser)

reader = csv.reader(open("ckpt/layers_info.csv", mode="r"))
layers = [{'inj_idx':inj_idx, 'layer_type':layer_type, 'model_idx':model_idx, 'layer_name':layer_name} for inj_idx, layer_type, model_idx, layer_name in reader]
results = json.load(open(f"ckpt/{args.name}_results.json", "r"))

In [ ]:
for i, (err, check) in results.items():
    if check < 1e-6:
        print(f"{i} {layers[int(i)]['layer_type']} \n  {layers[int(i)]['layer_name']} \n  {err}\n")
    else:
        raise ValueError(f"{layers[int(i)]} \n{err} \t {check}")

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

x = [int(i) for i, (err, _) in results.items() if layers[int(i)]['layer_type'] == "Conv2d"]
y = [err for i, (err, _) in results.items() if layers[int(i)]['layer_type'] == "Conv2d"]
plt.semilogy(x, y, label=f"Conv: {sum(y)/len(y):.2e}")
x = [int(i) - 1 for i, (err, _) in results.items() if layers[int(i)]['layer_type'] == "BatchNorm2d"]
y = [err for i, (err, _) in results.items() if layers[int(i)]['layer_type'] == "BatchNorm2d"]
plt.semilogy(x, y, label=f"BN: {sum(y)/len(y):.2e}")
# x = [int(i) for i, (err, _) in results.items()]
# y = [check for i, (_, check) in results.items()]
# plt.semilogy(x, y, label="BatchNorm2d")
plt.legend(loc="lower left")
plt.show()

* Short Conv module is very robust to injection
* BatchNorm2d is slightly less robust to injection than Conv2d

# Misc

In [ ]:
# From TIF file to numpy array

import numpy as np
import rasterio

img_path = "data/2021-09-01T10-00-00Z.tif"

# Open the images using rasterio
with rasterio.open(img_path) as img:
    b02 = img.read(2)  # Band 2 = Blue
    b03 = img.read(3)  # Band 3 = Green
    b04 = img.read(4)  # Band 4 = Red
    b08 = img.read(8)  # Band 8 = NIR
    b11 = img.read(11)  # Band 11 = SWIR1

    # Stack the bands
    img_image = np.stack([b02, b03, b04, b08, b11], axis=0).astype(np.float32)
# From TIF file to numpy array

import numpy as np
import rasterio

img_path = "data/2021-09-01T10-00-00Z.tif"

# Open the images using rasterio
with rasterio.open(img_path) as img:
    b02 = img.read(2)  # Band 2 = Blue
    b03 = img.read(3)  # Band 3 = Green
    b04 = img.read(4)  # Band 4 = Red
    b08 = img.read(8)  # Band 8 = NIR
    b11 = img.read(11)  # Band 11 = SWIR1

    # Stack the bands
    img_image = np.stack([b02, b03, b04, b08, b11], axis=0).astype(np.float32)